In [1]:
# Importing the necessary packages for data manipulation
import numpy as np
import pandas as pd

# Importing the necessary packages for data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Specifying preferences for the display of data
pd.options.display.max_columns=None #Show all columns
pd.options.display.max_rows=None #Show all rows

In [2]:
#Reading in the raw data
raw_data = pd.read_excel('Div in ABA survey data 9-6-23.xlsx', sheet_name="Sheet1")

In [3]:
# Dropping columns that won't be included in analyses
dropped_data = raw_data.drop(['Clinical_Influences', 'Finances_Influences', 'HR_Influences', \
                                  'Payor_Influences', 'Position_Title', 'Open_Comments' ], axis=1)

# Dropping rows that have more than 10 blank cells
trimmed_data = dropped_data.dropna(thresh=10) #drop rows if "na" in 10+ cells

In [4]:
#Turning raw text into binary features via one-hot encoding for the service setting column
#Specifying the new columns
clinic_outpatient=[]
community=[]
home=[]
hospital_inpatient=[]
residential_group_home=[]
school=[]
vocational_program=[]
other_setting=[]
setting_unknown=[]

#Specifying the text to include and exclude in each new column
for i in trimmed_data["Service_Settings"]:
    i = str (i).lower() #Turning all text to lowercase
    if "clinic" in i: 
        clinic_outpatient.append(1)
    else:
        clinic_outpatient.append(0)
    if ("community" in i) and ("retired from community" not in i):
        community.append(1)
    else:
        community.append(0)
    if ("home" in i) and ("residential/group home" not in i) and ("nursing home" not in i)\
    and ("retired from community, home and residential" not in i):
        home.append(1)
    else:
        home.append(0)
    if "hospital" in i:
        hospital_inpatient.append(1)
    else:
        hospital_inpatient.append(0)
    if "residential/group home" in i:
        residential_group_home.append(1)
    else:
        residential_group_home.append(0)
    if ("school" in i) or ("distric" in i):
        school.append(1)
    else:
        school.append(0)
    if "vocational program" in i:
        vocational_program.append(1)
    else:
        vocational_program.append(0)
    if ("telehealth" in i) or ("remote" in i) or ("daycare center" in i) or ("day cares" in i) \
    or ("day habs" in i) or ("adult day program" in i) or ("assisted living facilities" in i) \
    or ("memory care facilities" in i) or ("memory care facilities" in i) or ("nursing homes" in i) \
    or ("state supported living center" in i) or ("college" in i) or ("state government" in i) \
    or ("training crisis management" in i) or ("retired" in i):
        other_setting.append(1)
    else:
        other_setting.append(0)
    if "nan" in i:
        setting_unknown.append(1)
    else:
        setting_unknown.append(0)

#Converting the list to a single dataframe
service_settings_df = {
    'clinic_outpatient' : clinic_outpatient,
    'community' : community,
    'home' : home,
    'hospital_inpatient' : hospital_inpatient,
    'residential_group_home' : residential_group_home,
    'school' : school,
    'vocational_program' : vocational_program,
    'other_setting' : other_setting,
    'setting_unknown' : setting_unknown
}

service_settings_df = pd.DataFrame(service_settings_df)

#Adding an index column to allow for a clear merge with other dataframes later
service_settings_df['IDX'] = service_settings_df.index

In [5]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(service_settings_df):
    count_of_ones = service_settings_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / service_settings_df.count()) * 100  # Percentage of 1s
    settings_desc_df = pd.DataFrame({
        'Service Setting' : service_settings_df.columns,
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return settings_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
settings_desc_df = binary_count_and_percentage(service_settings_df)

#Drop IDX
settings_desc_df = settings_desc_df.drop('IDX')

# Display the result
print(settings_desc_df)

                               Service Setting  Count of 1s  Percentage of 1s
clinic_outpatient            clinic_outpatient          164         66.938776
school                                  school          111         45.306122
home                                      home          110         44.897959
community                            community          103         42.040816
residential_group_home  residential_group_home           38         15.510204
vocational_program          vocational_program           18          7.346939
other_setting                    other_setting           12          4.897959
hospital_inpatient          hospital_inpatient            3          1.224490
setting_unknown                setting_unknown            1          0.408163


In [6]:
#Ordinal encoding to transfer the org_size column text categories into ordinal values.
org_size=[]

#Specifying the ordinal values
size_mapping = {
    "1-10 employees": 1,
    "11-50 employees": 2,
    "51-200 employees": 3,
    "201-500 employees": 4,
    "501-1,000 employees": 5,
    "1,001-5,000 employees": 6,
    "5,001-10,000 employees" : 7,
    "10,000+ employees" : 8
}

#Loop through to encode the above
for i in trimmed_data["N_Employees"]:
    org_size.append(size_mapping.get(i, -1))

#Creating a new dataframe with the ordinal values
org_size_df = {
    'org_size' : org_size,

}
    
org_size_df = pd.DataFrame(org_size_df)

#Adding an index column for future use
org_size_df['IDX'] = org_size_df.index

In [7]:
# Function to calculate both count and percentage occurrence of each number
def count_and_percentage_occurrence(org_size):
    count = org_size.value_counts()  # Get counts
    percentage = (count / len(org_size)) * 100  # Calculate percentages
    org_size_desc_df = pd.DataFrame({'Count': count, 'Percentage': percentage})
    return org_size_desc_df.sort_values(by='Count', ascending=False)

# Call the function
org_size_desc_df = count_and_percentage_occurrence(org_size_df['org_size'])

# Display the count and percentage occurrence for each number
print(org_size_desc_df)

   Count  Percentage
2     80   32.653061
3     53   21.632653
4     36   14.693878
1     27   11.020408
5     19    7.755102
6     18    7.346939
7      7    2.857143
8      5    2.040816


In [8]:
#Converting text to binary values via one-hot encoding for the geographic region column
#Specifying the new columns
midwest_us=[]
south_us=[]
west_us=[]
northeast_us=[]
other_geo=[]
geo_unknown=[]

#Specifying the text to include and exclude for each new column
for i in trimmed_data["Geo_Region"]:
    i = str (i).lower()
    if "nd, sd, ne, ks" in i: 
        midwest_us.append(1)
    else:
        midwest_us.append(0)
    if "de, md, wv, va, ky" in i:
        south_us.append(1)
    else:
        south_us.append(0)
    if "wa, or, id, mt, wy" in i:
        west_us.append(1)
    else:
        west_us.append(0)
    if "me, vt, nh, ma" in i:
        northeast_us.append(1)
    else:
        northeast_us.append(0)
    if ("british columbia" in i) or ("canada" in i) or ("ontario" in i) or ("internationally" in i) \
    or ("middle east" in i) or ("saudi arabia" in i) or ("riyadh" in i) or ("brazil" in i) or ("usvi" in i) \
    or ("brazil" in i):
        other_geo.append(1)
    else:
        other_geo.append(0)
    if "nan" in i:
        geo_unknown.append(1)
    else:
        geo_unknown.append(0)

#Creating the new dataframe with the binary data
geo_df = {
    'midwest_us' : midwest_us,
    'south_us' : south_us,
    'west_us' : west_us,
    'northeast_us' : northeast_us,
    'other_geo' : other_geo,
    'geo_unknown' : geo_unknown

}

geo_df = pd.DataFrame(geo_df)

#Adding an index column for future use
geo_df['IDX'] = geo_df.index

In [9]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(geo_df):
    count_of_ones = geo_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / geo_df.count()) * 100  # Percentage of 1s
    geo_desc_df = pd.DataFrame({
        'Geo Region' : geo_df.columns,
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return geo_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
geo_desc_df = binary_count_and_percentage(geo_df)

#drop the IDX
geo_desc_df = geo_desc_df.drop ('IDX')

# Display the result
print(geo_desc_df)

                Geo Region  Count of 1s  Percentage of 1s
south_us          south_us           96         39.183673
midwest_us      midwest_us           64         26.122449
northeast_us  northeast_us           64         26.122449
west_us            west_us           63         25.714286
other_geo        other_geo           12          4.897959
geo_unknown    geo_unknown            1          0.408163


In [10]:
#Converting text to binary values for the ceo background column
#Specifying the new columns
ceo_behavior_analysis=[]
ceo_business=[]
ceo_education=[]
ceo_finance=[]
ceo_psychology=[]
ceo_human_resources=[]
ceo_organizational_behavior_management=[]
ceo_other_allied=[]
ceo_other_other=[]
ceo_unknown=[]

#Specifying what to include or exclude in the new columns
for i in trimmed_data["CEO_Background"]:
    i = str (i).lower()
    if ("analysis" in i) or ("bcba" in i): 
        ceo_behavior_analysis.append(1)
    else:
        ceo_behavior_analysis.append(0)
    if "business" in i:
        ceo_business.append(1)
    else:
        ceo_business.append(0)
    if ("education" in i) or ("m.ed." in i):
        ceo_education.append(1)
    else:
        ceo_education.append(0)
    if "finance" in i:
        ceo_finance.append(1)
    else:
        ceo_finance.append(0)
    if "resources" in i:
        ceo_human_resources.append(1)
    else:
        ceo_human_resources.append(0)
    if "psychology" in i:
        ceo_psychology.append(1)
    else:
        ceo_psychology.append(0)
    if "organizational behavior" in i:
        ceo_organizational_behavior_management.append(1)
    else:
        ceo_organizational_behavior_management.append(0)
    if ("healthcare" in i) or ("health care" in i) or ("behavioral health" in i) or ("public health" in i) \
    or ("speech" in i) or ("occupational therapy" in i) or ("social work" in i) or ("counseling" in i) \
    or ("md" in i) or ("physician" in i) or ("neurology" in i): 
        ceo_other_allied.append(1)
    else:
        ceo_other_allied.append(0)
    if ("sociology" in i) or ("law" in i) or ("politics" in i) or ("political science" in i) \
    or ("public school" in i) or ("university" in i) or ("administration" in i):
        ceo_other_other.append(1)
    else:
        ceo_other_other.append(0)
    if ("nan" in i) or ("i don't know" in i):
        ceo_unknown.append(1)
    else:
        ceo_unknown.append(0)

#Creating the new dataframe
ceo_background_frame = {
    'ceo_behavior_analysis' : ceo_behavior_analysis,
    'ceo_business' : ceo_business,
    'ceo_education' : ceo_education,
    'ceo_finance' : ceo_finance,
    'ceo_psychology' : ceo_psychology,
    'ceo_human_resources' : ceo_human_resources,
    'ceo_organizational_behavior_management' : ceo_organizational_behavior_management,
    'ceo_other_allied' : ceo_other_allied,
    'ceo_other_other' : ceo_other_other,
    'ceo_unknown' : ceo_unknown
}

ceo_df = pd.DataFrame(ceo_background_frame)

#Creating an index column for future use
ceo_df['IDX'] = ceo_df.index  

In [11]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(ceo_df):
    count_of_ones = ceo_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / ceo_df.count()) * 100  # Percentage of 1s
    ceo_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return ceo_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
ceo_desc_df = binary_count_and_percentage(ceo_df)

# Display the result
print(ceo_desc_df)

                                        Count of 1s  Percentage of 1s
IDX                                           29890      12200.000000
ceo_behavior_analysis                           130         53.061224
ceo_business                                     81         33.061224
ceo_psychology                                   46         18.775510
ceo_education                                    42         17.142857
ceo_other_allied                                 23          9.387755
ceo_unknown                                      15          6.122449
ceo_organizational_behavior_management           14          5.714286
ceo_finance                                      13          5.306122
ceo_other_other                                  10          4.081633
ceo_human_resources                               4          1.632653


In [12]:
#Converting text to binary values for the current position column
#Specifying the new columns
board_member=[]
case_manager=[]
director=[]
direct_service=[]
executive_member=[]
position_other=[]
position_unknown=[]

#Specifying what to include and exclude for each new column
for i in trimmed_data["Position_Category"]:
    i = str (i).lower()
    if "board member" in i: 
        board_member.append(1)
    else:
        board_member.append(0)
    if ("case manager" in i) or ("clinical;" in i) or ("clinical," in i) \
    or ("clinician" in i) or ("practitioner" in i) or ("supervisor" in i):
        case_manager.append(1)
    else:
        case_manager.append(0)
    if ("department director" in i) or ("regional or clinic director" in i) or ("regional coordinator" in i):
        director.append(1)
    else:
        director.append(0)
    if ("direct service" in i) or ("rbt" in i):
        direct_service.append(1)
    else:
        direct_service.append(0)
    if ("executive member" in i) or ("chief clinical officer" in i) or ("avp" in i) or ("owner" in i):
        executive_member.append(1)
    else:
        executive_member.append(0)
    if ("bureaucrat" in i) or ("slt member" in i) or ("management" in i) or ("teacher" in i) \
    or ("consultant" in i) or ("research" in i) or ("trainer" in i) or ("admin" in i):
        position_other.append(1)
    else:
        position_other.append(0)
    if "nan" in i:
        position_unknown.append(1)
    else:
        position_unknown.append(0)

#Creating the new dataframe with the binary values
position_category_frame = {
    'board_member' : board_member,
    'case_manager' : case_manager,
    'director' : director,
    'direct_service' : direct_service,
    'executive_member' : executive_member,
    'position_other' : position_other,
    'position_unknown' : position_unknown

}

position_category_df = pd.DataFrame(position_category_frame)

#Adding an index column for future use
position_category_df['IDX'] = position_category_df.index

In [13]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(position_category_df):
    count_of_ones = position_category_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / position_category_df.count()) * 100  # Percentage of 1s
    position_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return position_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
position_desc_df = binary_count_and_percentage(position_category_df)

# Display the result
print(position_desc_df)

                  Count of 1s  Percentage of 1s
IDX                     29890      12200.000000
director                  105         42.857143
case_manager               83         33.877551
executive_member           59         24.081633
direct_service             20          8.163265
position_other             13          5.306122
position_unknown            6          2.448980
board_member                4          1.632653


In [14]:
#Ordinal encoding for the position duration column to convert the categorical text to ordinal values.
position_duration=[]

#Specifying the values for encoding
years_mapping = {
    "1 year or less": 1,
    "1-5 years" : 2,
    "2-5 years": 2,
    "6-10 years": 3,
    "11-15 years": 4,
    "More than 15 years": 5,

}

#Looping through the column to encode
for i in trimmed_data["Position_Duration"]:
    position_duration.append(years_mapping.get(i, -1))

#Creating a new dataframe for the ordinal values
position_duration_df = {
    'position_duration' : position_duration,

}
    
position_duration_df = pd.DataFrame(position_duration_df)

#Adding an index column for future use
position_duration_df['IDX'] = position_duration_df.index

In [15]:
# Function to calculate both count and percentage occurrence of each number
def count_and_percentage_occurrence(position_duration):
    count = position_duration.value_counts()  # Get counts
    percentage = (count / len(position_duration)) * 100  # Calculate percentages
    pos_dur_desc_df = pd.DataFrame({'Count': count, 'Percentage': percentage})
    return pos_dur_desc_df.sort_values(by='Count', ascending=False)

# Call the function
pos_dur_desc_df = count_and_percentage_occurrence(position_duration_df['position_duration'])

# Display the count and percentage occurrence for each number
print(pos_dur_desc_df)

    Count  Percentage
 2    130   53.061224
 3     44   17.959184
 1     42   17.142857
 4     19    7.755102
 5      9    3.673469
-1      1    0.408163


In [16]:
#Ordinal encoding for the education/degree column
degree_ordinal = []

#Specifying the ordinal values
degree_mapping = {
    "bachelor's degree\xa0": 1,
    "master's degree\xa0": 2,
    "doctoral degree\xa0": 3
}

#Looping through the education column for encoding
for i in trimmed_data["Education"]:
    if pd.isna(i):  # Check for NaN values
        degree_ordinal.append(0)  # Append the mapped value for NaN
    else:
        i = str(i).lower()
        degree_ordinal.append(degree_mapping.get(i, -1))  # Default to -1 if not found

#Creating a new dataframe with the ordinal values
degree_ordinal_df = {
    'degree_ordinal': degree_ordinal
}

degree_ordinal_df = pd.DataFrame(degree_ordinal_df)

#Adding an index column for future use
degree_ordinal_df['IDX'] = degree_ordinal_df.index

In [17]:
# Function to calculate both count and percentage occurrence of each number
def count_and_percentage_occurrence(degree_ordinal_df):
    count = degree_ordinal_df.value_counts()  # Get counts
    percentage = (count / len(degree_ordinal_df)) * 100  # Calculate percentages
    degree_desc_df = pd.DataFrame({'Count': count, 'Percentage': percentage})
    return degree_desc_df.sort_values(by='Count', ascending=False)

# Call the function
degree_desc_df = count_and_percentage_occurrence(degree_ordinal_df['degree_ordinal'])

# Display the count and percentage occurrence for each number
print(degree_desc_df)

    Count  Percentage
 2    169   68.979592
 3     52   21.224490
 1     14    5.714286
-1      9    3.673469
 0      1    0.408163


In [18]:
#Converting text to binary values for the education domain column
#Specifying the new columns
ed_behavior_analysis=[]
ed_business=[]
ed_education=[]
ed_finance=[]
ed_hr=[]
ed_obm=[]
ed_psychology=[]
ed_social_work=[]
ed_domain_other=[]
ed_domain_unknown=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Training_Domain"]:
    i = str (i).lower()
    if ("behavior analysis" in i) or ("beh anal" in i): 
        ed_behavior_analysis.append(1)
    else:
        ed_behavior_analysis.append(0)
    if "business" in i:
        ed_business.append(1)
    else:
        ed_business.append(0)
    if "education" in i:
        ed_education.append(1)
    else:
        ed_education.append(0)
    if "finance" in i:
        ed_finance.append(1)
    else:
        ed_finance.append(0)
    if "human resources" in i:
        ed_hr.append(1)
    else:
        ed_hr.append(0)
    if "organizational behavior management" in i:
        ed_obm.append(1)
    else:
        ed_obm.append(0)
    if "psychology" in i:
        ed_psychology.append(1)
    else:
        ed_psychology.append(0)
    if "social work" in i:
        ed_social_work.append(1)
    else:
        ed_social_work.append(0)
    if ("recreational therapy" in i) or ("health promotion" in i) or ("organizational leadership" in i)\
    or ("counseling" in i) or ("sociology" in i) or ("neuroscience" in i) or ("journalism" in i)\
    or ("political science" in i) or ("public service" in i) or ("philosophy and theology" in i)\
    or ("communication sciences and disorders" in i):
        ed_domain_other.append(1)
    else:
        ed_domain_other.append(0)
    if "nan" in i:
        ed_domain_unknown.append(1)
    else:
        ed_domain_unknown.append(0)

#Creating a new dataframe with the binary values
training_df = {
    'ed_behavior_analysis' : ed_behavior_analysis,
    'ed_business' : ed_business,
    'ed_education' : ed_education,
    'ed_finance' : ed_finance,
    'ed_hr' : ed_hr,
    'ed_obm' : ed_obm,
    'ed_psychology' : ed_psychology,
    'ed_social_work' : ed_social_work,
    'ed_domain_other' : ed_domain_other,
    'ed_domain_unknown' : ed_domain_unknown

}

training_df = pd.DataFrame(training_df)

#Adding an index column for future use
training_df['IDX'] = training_df.index

In [19]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(training_df):
    count_of_ones = training_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / training_df.count()) * 100  # Percentage of 1s
    train_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return train_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
train_desc_df = binary_count_and_percentage(training_df)

# Display the result
print(train_desc_df)

                      Count of 1s  Percentage of 1s
IDX                         29890      12200.000000
ed_behavior_analysis          236         96.326531
ed_psychology                 119         48.571429
ed_education                   98         40.000000
ed_obm                         47         19.183673
ed_domain_other                16          6.530612
ed_business                    15          6.122449
ed_social_work                 10          4.081633
ed_hr                           7          2.857143
ed_finance                      6          2.448980
ed_domain_unknown               6          2.448980


In [20]:
#Converting text to binary values for the certification column
#Specifying the new columns
bcba=[]
cpa=[]
slp=[]
lba=[]
licensed_psychologist=[]
licensed_sw=[]
other_rbt_bcaba=[]
other_cert=[]
other_cert_teaching=[]
no_cert=[]
cert_unknown=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Certifications"]:
    i = str (i).lower()
    if "board certified behavior analyst" in i: 
        bcba.append(1)
    else:
        bcba.append(0)
    if ("certified public accountant" in i) or ("lpa" in i):
        cpa.append(1)
    else:
        cpa.append(0)
    if "ccc-slp" in i:
        slp.append(1)
    else:
        slp.append(0)
    if "licensed behavior analyst" in i:
        lba.append(1)
    else:
        lba.append(0)
    if ("licensed psychologist" in i) or ("limited license psychologist" in i) or ("school psychologist" in i)\
    or ("lssp" in i):
        licensed_psychologist.append(1)
    else:
        licensed_psychologist.append(0)
    if "licensed social worker" in i:
        licensed_sw.append(1)
    else:
        licensed_sw.append(0)
    if ("rbt" in i) or ("registered behavior technician" in i) or ("board certified assistant behavior analyst" in i):
        other_rbt_bcaba.append(1)
    else:
        other_rbt_bcaba.append(0)
    if ("certified special educator" in i) or ("certified teacher" in i) or ("prek-12 license" in i) \
    or ("teaching license" in i) or ("licensed educator" in i) or ("special education experienced educator" in i) \
    or ("special education teacher" in i) or ("special education teaching" in i) or ("licensed sped teacher" in i) \
    or ("licensed special educator" in i) or ("teacher certification" in i) or ("education specialist" in i) or\
    ("educational assistant" in i) or ("sp ed teacher" in i):
        other_cert_teaching.append(1)
    else:
        other_cert_teaching.append(0)
    if ("laba" in i) or ("certified ohio behavior analyst" in i) or ("licensed behavior specialist" in i)\
    or ("licensed recreational therapist" in i) or ("certified therapeutic recreation specialist" in i)\
    or ("certified autism specialist" in i) or ("qasp-s" in i) or ("certified scrum master" in i) \
    or ("iba" in i) or ("international behavior analyst" in i) or ("lmhc" in i) or ("lpc" in i) \
    or ("mental health counselor" in i):
        other_cert.append(1)
    else:
        other_cert.append(0)
    if "no certificate" in i:
        no_cert.append(1)
    else:
        no_cert.append(0)
    if "nan" in i:
        cert_unknown.append(1)
    else:
        cert_unknown.append(0)

#Creating a new dataframe with the binary values
certifications_df = {
    'bcba' : bcba,
    'cpa' : cpa,
    'slp' : slp,
    'lba' : lba,
    'licensed_psychologist' : licensed_psychologist,
    'licensed_sw' : licensed_sw,
    'other_rbt_bcaba' : other_rbt_bcaba,
    'other_cert_teaching' : other_cert_teaching,
    'other_cert' : other_cert,
    'no_cert' : no_cert,
    'cert_unknown' : cert_unknown

}

certifications_df = pd.DataFrame(certifications_df)

#Adding an index column for future use
certifications_df['IDX'] = certifications_df.index

In [21]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(certifications_df):
    count_of_ones = certifications_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / certifications_df.count()) * 100  # Percentage of 1s
    cert_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return cert_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
cert_desc_df = binary_count_and_percentage(certifications_df)

# Display the result
print(cert_desc_df)

                       Count of 1s  Percentage of 1s
IDX                          29890      12200.000000
bcba                           214         87.346939
lba                            116         47.346939
licensed_psychologist           22          8.979592
other_cert_teaching             18          7.346939
other_rbt_bcaba                 15          6.122449
other_cert                      15          6.122449
cert_unknown                     8          3.265306
slp                              2          0.816327
licensed_sw                      2          0.816327
cpa                              1          0.408163
no_cert                          1          0.408163


In [22]:
#Converting categorical text to ordinal values for the age column
age_ordinal=[]

#Specifying the ordinal values
age_mapping = {
    "18-24 years old": 1,
    "25-34 years old" : 2,
    "35-44 years old": 3,
    "45-54 years old": 4,
    "55-64 years old": 5,
    "65 years or older": 6,
    "prefer not to answer" : 0

}

#Looping through the column for encoding
for i in trimmed_data["Age"]:
    age_ordinal.append(age_mapping.get(i, -1))

#Creating a new dataframe with the ordinal values
age_ordinal_df = {
    'age_ordinal' : age_ordinal

}
    
age_ordinal_df = pd.DataFrame(age_ordinal_df)

#Adding an index column for future use
age_ordinal_df['IDX'] = age_ordinal_df.index

In [23]:
# Function to calculate both count and percentage occurrence of each number
def count_and_percentage_occurrence(age_ordinal_df):
    count = age_ordinal_df.value_counts()  # Get counts
    percentage = (count / len(age_ordinal_df)) * 100  # Calculate percentages
    age_desc_df = pd.DataFrame({'Count': count, 'Percentage': percentage})
    return age_desc_df.sort_values(by='Count', ascending=False)

# Call the function
age_desc_df = count_and_percentage_occurrence(age_ordinal_df['age_ordinal'])

# Display the count and percentage occurrence for each number
print(age_desc_df)

    Count  Percentage
 3    100   40.816327
 2     61   24.897959
 4     46   18.775510
 5     22    8.979592
 6      9    3.673469
 1      6    2.448980
-1      1    0.408163


In [24]:
#Converting text to binary values for the race column
#Specifying the new columns
american_indian_or_alaska_native=[]
black_or_african_american=[]
white=[]
middle_eastern=[]
arab=[]
asian=[]
hispanic_or_latinx=[]
native_hawaiian_or_pacific_islander=[]
race_mixed=[]
race_prefer_not_to_answer=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Race"]:
    i = str (i).lower()
    if ("indian" in i) or ("native american" in i): 
        american_indian_or_alaska_native.append(1)
    else:
        american_indian_or_alaska_native.append(0)
    if "black" in i:
        black_or_african_american.append(1)
    else:
        black_or_african_american.append(0)
    if "white" in i:
        white.append(1)
    else:
        white.append(0)
    if "middle eastern" in i:
        middle_eastern.append(1)
    else:
        middle_eastern.append(0)
    if "arab" in i:
        arab.append(1)
    else:
        arab.append(0)
    if "asian" in i:
        asian.append(1)
    else:
        asian.append(0)
    if "hispanic" in i:
        hispanic_or_latinx.append(1)
    else:
        hispanic_or_latinx.append(0)
    if "islander" in i:
        native_hawaiian_or_pacific_islander.append(1)
    else:
        native_hawaiian_or_pacific_islander.append(0)
    if "mixed" in i: 
        race_mixed.append(1)
    else:
        race_mixed.append(0)
    if "prefer" in i:
        race_prefer_not_to_answer.append(1)
    else:
        race_prefer_not_to_answer.append(0)

#Creating a new dataframe with the binary values       
race_df = {
    'american_indian_or_alaska_native' : american_indian_or_alaska_native,
    'black_or_african_american' : black_or_african_american,
    'white' : white,
    'middle_eastern' : middle_eastern,
    'arab' : arab,
    'asian' : asian,
    'hispanic_or_latinx' : hispanic_or_latinx,
    'native_hawaiian_or_pacific_islander' : native_hawaiian_or_pacific_islander,
    'race_mixed' : race_mixed,
    'race_prefer_not_to_answer' : race_prefer_not_to_answer,
}

race_df = pd.DataFrame(race_df)

#Adding an index column for future merging use
race_df['IDX'] = race_df.index

In [25]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(race_df):
    count_of_ones = race_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / race_df.count()) * 100  # Percentage of 1s
    race_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return race_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
race_desc_df = binary_count_and_percentage(race_df)

# Display the result
print(race_desc_df)

                                     Count of 1s  Percentage of 1s
IDX                                        29890      12200.000000
white                                        206         84.081633
hispanic_or_latinx                            18          7.346939
asian                                         16          6.530612
black_or_african_american                     15          6.122449
american_indian_or_alaska_native               3          1.224490
middle_eastern                                 2          0.816327
race_prefer_not_to_answer                      2          0.816327
arab                                           1          0.408163
native_hawaiian_or_pacific_islander            1          0.408163
race_mixed                                     1          0.408163


In [26]:
#Converting text to binary values for the gender column
#Specifying the new columns
nonbinary=[]
cisgender_female=[]
cisgender_male=[]
female=[]
transgender_female=[]
transgender_male=[]
gender_prefer_no_answer=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Gender"]:
    i = str (i).lower()
    if "nonbinary" in i: 
        nonbinary.append(1)
    else:
        nonbinary.append(0)
    if "cisgender female" in i:
        cisgender_female.append(1)
    else:
        cisgender_female.append(0)
    if "cisgender male" in i:
        cisgender_male.append(1)
    else:
        cisgender_male.append(0)
    if ("female" in i) and ("cisgender female" not in i) and ("transgender female" not in i):
        female.append(1)
    else:
        female.append(0)
    if "transgender female" in i:
        transgender_female.append(1)
    else:
        transgender_female.append(0)
    if "transgender male" in i:
        transgender_male.append(1)
    else:
        transgender_male.append(0)
    if "prefer not to answer" in i:
        gender_prefer_no_answer.append(1)
    else:
        gender_prefer_no_answer.append(0)

#Creating a new dataframe with the binary values
gender_df = {
    'nonbinary' : nonbinary,
    'cisgender_female' : cisgender_female,
    'cisgender_male' : cisgender_male,
    'female' : female,
    'transgender_female' : transgender_female,
    'transgender_male' : transgender_male,
    'gender_prefer_no_answer' : gender_prefer_no_answer,

}

gender_df = pd.DataFrame(gender_df)

#Adding an index column for future use
gender_df['IDX'] = gender_df.index

In [27]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(gender_df):
    count_of_ones = gender_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / gender_df.count()) * 100  # Percentage of 1s
    gender_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return gender_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
gender_desc_df = binary_count_and_percentage(gender_df)

# Display the result
print(gender_desc_df)

                         Count of 1s  Percentage of 1s
IDX                            29890      12200.000000
cisgender_female                 200         81.632653
cisgender_male                    29         11.836735
female                             8          3.265306
gender_prefer_no_answer            6          2.448980
nonbinary                          1          0.408163
transgender_female                 1          0.408163
transgender_male                   0          0.000000


In [28]:
#Converting text to binary values for the sexual orientation column
#Specifying the new columns
asexual=[]
bisexual=[]
heterosexual=[]
homosexual=[]
pansexual=[]
sex_prefer_no_answer=[]
other_queer=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Sexual_Orientation"]:
    i = str (i).lower()
    if "asexual" in i: 
        asexual.append(1)
    else:
        asexual.append(0)
    if "bisexual" in i:
        bisexual.append(1)
    else:
        bisexual.append(0)
    if "heterosexual" in i:
        heterosexual.append(1)
    else:
        heterosexual.append(0)
    if "homosexual" in i:
        homosexual.append(1)
    else:
        homosexual.append(0)
    if "pansexual" in i:
        pansexual.append(1)
    else:
        pansexual.append(0)
    if "prefer not to answer" in i:
        sex_prefer_no_answer.append(1)
    else:
        sex_prefer_no_answer.append(0)
    if "queer" in i:
        other_queer.append(1)
    else:
        other_queer.append(0)

#Creating a new dataframe with the binary values
sex_df = {
    'asexual' : asexual,
    'bisexual' : bisexual,
    'heterosexual' : heterosexual,
    'homosexual' : homosexual,
    'pansexual' : pansexual,
    'sex_prefer_no_answer' : sex_prefer_no_answer,
    'other_queer' : other_queer,
    
}

sex_df = pd.DataFrame(sex_df)

#Adding an index column for future use
sex_df['IDX'] = sex_df.index

In [29]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(sex_df):
    count_of_ones = sex_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / sex_df.count()) * 100  # Percentage of 1s
    sex_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return sex_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
sex_desc_df = binary_count_and_percentage(sex_df)

# Display the result
print(sex_desc_df)

                      Count of 1s  Percentage of 1s
IDX                         29890      12200.000000
heterosexual                  192         78.367347
bisexual                       18          7.346939
sex_prefer_no_answer           11          4.489796
homosexual                      9          3.673469
asexual                         7          2.857143
pansexual                       7          2.857143
other_queer                     1          0.408163


In [30]:
#Converting text to binary values for the religion column
#Specifying the new columns
buddhism=[]
christianity=[]
hinduism=[]
indigenous_religion=[]
islam=[]
judaism=[]
non_religious=[]
religion_other=[]
religion_prefer_no_answer=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Religious_Affiliation"]:
    i = str (i).lower()
    if ("agnost" in i) or ("atheis" in i) or ("spiritual" in i): 
        religion_other.append(1)
    else:
        religion_other.append(0)

    if "buddhis" in i:
        buddhism.append(1)
    else:
        buddhism.append(0)
    if ("cathol" in i) or ("christianity" in i):
        christianity.append(1)
    else:
        christianity.append(0)
    if "hinduism" in i:
        hinduism.append(1)
    else:
        hinduism.append(0)
    if "indigenous" in i:
        indigenous_religion.append(1)
    else:
        indigenous_religion.append(0)
    if "islam" in i:
        islam.append(1)
    else:
        islam.append(0)
    if "judaism" in i:
        judaism.append(1)
    else:
        judaism.append(0)
    if "non" in i:
        non_religious.append(1)
    else:
        non_religious.append(0)
    if "pref" in i:
        religion_prefer_no_answer.append(1)
    else:
        religion_prefer_no_answer.append(0)

#Creating a new dataframe with the binary values
religious_affiliation_frame = {
    'buddhism' : buddhism,
    'christianity' : christianity,
    'hinduism' : hinduism,
    'indigenous_religion' : indigenous_religion,
    'islam' : islam,
    'judaism' : judaism,
    'religion_other' : religion_other,
    'non_religious' : non_religious,
    'religion_prefer_no_answer' : religion_prefer_no_answer,
}

ra_df = pd.DataFrame(religious_affiliation_frame)

#Creating an index column for future use
ra_df['IDX'] = ra_df.index

In [31]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(ra_df):
    count_of_ones = ra_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / ra_df.count()) * 100  # Percentage of 1s
    ra_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return ra_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
ra_desc_df = binary_count_and_percentage(ra_df)

# Display the result
print(ra_desc_df)

                           Count of 1s  Percentage of 1s
IDX                              29890      12200.000000
non_religious                      116         47.346939
christianity                       101         41.224490
religion_prefer_no_answer           13          5.306122
judaism                             10          4.081633
religion_other                       9          3.673469
islam                                4          1.632653
hinduism                             2          0.816327
buddhism                             1          0.408163
indigenous_religion                  1          0.408163


In [32]:
#Converting text to binary values for the veteran status column
#Specifying the new columns
protected_vet=[]
not_protected_vet=[]
other_family=[]
vet_prefer_not_answer=[]
vet_unknown=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Veteran_Status"]:
    i = str (i).lower()
    if ("i am a disabled veteran" in i) or ("i am a recently separated veteran" in i) or \
    ("i am an active duty wartime" in i) or ("i am an armed forces service medal veteran" in i) \
    or ("i am a va army national guard veteran" in i) or ("honorable discharge" in i): 
        protected_vet.append(1)
    else:
        protected_vet.append(0)
    if ("i am not a protected veteran" in i) or ("military spouse" in i) or ("veteran spouse" in i) \
    or ("spouse of veteran" in i) or ("wife of a veteran" in i) or ("dependent of active duty" in i) \
    or ("veteran from a foreign arm" in i) or ("i am not a veteran" in i) or ("none" in i) or \
    ("not a veteran" in i) or ("not veteran" in i):
        not_protected_vet.append(1)
    else:
        not_protected_vet.append(0)
    if ("military spouse" in i) or ("veteran spouse" in i) or ("spouse of veteran" in i) \
    or ("wife of a veteran" in i) or ("dependent of active duty" in i):
        other_family.append(1)
    else:
        other_family.append(0)
    if "prefer not to answer" in i:
        vet_prefer_not_answer.append(1)
    else:
        vet_prefer_not_answer.append(0)
    if "nan" in i:
        vet_unknown.append(1)
    else:
        vet_unknown.append(0)

#Creating a new dataframe with the binary values
vet_df = {
    'protected_vet' : protected_vet,
    'not_protected_vet' : not_protected_vet,
    'other_family' : other_family,
    'vet_prefer_not_answer' : vet_prefer_not_answer,
    'vet_unknown' : vet_unknown
}

vet_df = pd.DataFrame(vet_df)

#Adding an index column for future use
vet_df['IDX'] = vet_df.index

In [33]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(vet_df):
    count_of_ones = vet_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / vet_df.count()) * 100  # Percentage of 1s
    vet_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return vet_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
vet_desc_df = binary_count_and_percentage(vet_df)

# Display the result
print(vet_desc_df)

                       Count of 1s  Percentage of 1s
IDX                          29890      12200.000000
not_protected_vet              231         94.285714
protected_vet                    7          2.857143
other_family                     5          2.040816
vet_unknown                      4          1.632653
vet_prefer_not_answer            3          1.224490


In [34]:
#Converting text to binary values for the disability column
#Specifying the new columns
no_disability=[]
yes_disability=[]
other_invisible_disability=[]
dis_prefer_no_answer=[]
disability_unknown=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Disability_Status"]:
    i = str (i).lower()
    if "i do not have a disability" in i: 
        no_disability.append(1)
    else:
        no_disability.append(0)
    if ("i have a disability" in i) or ("invisible disability" in i):
        yes_disability.append(1)
    else:
        yes_disability.append(0)
    if "invisible disability" in i:
        other_invisible_disability.append(1)
    else:
        other_invisible_disability.append(0)
    if "prefer not to answer" in i:
        dis_prefer_no_answer.append(1)
    else:
        dis_prefer_no_answer.append(0)
    if "nan" in i:
        disability_unknown.append(1)
    else:
        disability_unknown.append(0)
  
#Creating a new dataframe with the binary values
disability_df = {
    'no_disability' : no_disability,
    'yes_disability' : yes_disability,
    'other_invisible_disability' : other_invisible_disability,
    'dis_prefer_no_answer' : dis_prefer_no_answer,
    'disability_unknown' : disability_unknown
    
}

disability_df = pd.DataFrame(disability_df)

#Adding an index column for future use
disability_df['IDX'] = disability_df.index

In [35]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(disability_df):
    count_of_ones = disability_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / disability_df.count()) * 100  # Percentage of 1s
    dis_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return dis_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
dis_desc_df = binary_count_and_percentage(disability_df)

# Display the result
print(dis_desc_df)

                            Count of 1s  Percentage of 1s
IDX                               29890      12200.000000
no_disability                       207         84.489796
yes_disability                       28         11.428571
dis_prefer_no_answer                  7          2.857143
disability_unknown                    3          1.224490
other_invisible_disability            1          0.408163


In [36]:
#Creating a dataframe for binary values for minority for each demographic category
#Specifying the new columns
female=[]
race_minority=[]
age_minority=[]
other_gender_minority=[]
sex_minority=[]
religion_minority=[]
vet_minority=[]
dis_minority=[]

#Specifying what to include and exclude for the binary values for each new column
for i in trimmed_data["Gender"]:
    i = str (i).lower()
    if ("cisgender female" in i) or ("female" in i) and not ("transgender female" in i):
        female.append(1)
    else:
        female.append(0)
for i in trimmed_data["Race"]:
    i = str (i).lower()
    if ("indian" in i) or ("native american" in i) or ("black" in i) or ("middle eastern" in i) \
    or ("arab" in i) or ("asian" in i) or ("hispanic" in i) or ("islander" in i) or ("mixed" in i):
        race_minority.append(1)
    else:
        race_minority.append(0)
for i in trimmed_data["Gender"]:
    i = str (i).lower()
    if ("nonbinary" in i) or ("transgender female" in i):
        other_gender_minority.append(1)
    else:
        other_gender_minority.append(0)
for i in trimmed_data["Sexual_Orientation"]:
    i = str (i).lower()
    if ("asexual" in i) or ("bisexual" in i) or ("homosexual" in i) or ("pansexual" in i) or ("queer" in i):
        sex_minority.append(1)
    else:
        sex_minority.append(0)
for i in trimmed_data["Religious_Affiliation"]:
    i = str (i).lower()
    if ("agnost" in i) or ("atheis" in i) or ("buddhis" in i) or ("hinduism" in i) or ("indigenous" in i) \
    or ("islam" in i) or ("judaism" in i) or ("spiritual" in i) or ("non" in i): 
        religion_minority.append(1)
    else:
        religion_minority.append(0)
for i in trimmed_data["Veteran_Status"]:
    i = str (i).lower()
    if ("i am a disabled veteran" in i) or ("i am a recently separated veteran" in i) or \
    ("i am an active duty wartime" in i) or ("i am an armed forces service medal veteran" in i) \
    or ("i am a va army national guard veteran" in i) or ("honorable discharge" in i) \
    or ("military spouse" in i) or ("veteran spouse" in i) or ("spouse of veteran" in i) \
    or ("wife of a veteran" in i) or ("dependent of active duty" in i): 
        vet_minority.append(1)
    else:
        vet_minority.append(0)
for i in trimmed_data["Disability_Status"]:
    i = str (i).lower()
    if ("i have a disability" in i) or ("invisible disability" in i):
        dis_minority.append(1)
    else:
        dis_minority.append(0)

#Creating a new dataframe with the binary values
diversity_df = {
    'female' : female,
    'race_minority' : race_minority,
    'other_gender_minority' : other_gender_minority,
    'sex_minority' : sex_minority,
    'religion_minority' : religion_minority,
    'vet_minority' : vet_minority,
    'dis_minority' : dis_minority,

}

diversity_df = pd.DataFrame(diversity_df)

In [37]:
#Adding an age minority column with binary values for the ordinal values protected by age discrimination laws.
diversity_df['age_minority'] = age_ordinal_df['age_ordinal'].apply(lambda x: 1 if x in [4, 5, 6] else 0)

In [38]:
# Function to calculate count and percentage of 1s for each column
def binary_count_and_percentage(diviersity_df):
    count_of_ones = diversity_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / diversity_df.count()) * 100  # Percentage of 1s
    div_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return div_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
div_desc_df = binary_count_and_percentage(disability_df)

# Display the result
print(div_desc_df)

                       Count of 1s  Percentage of 1s
female                         208         84.897959
religion_minority              135         55.102041
age_minority                    77         31.428571
race_minority                   52         21.224490
sex_minority                    42         17.142857
dis_minority                    28         11.428571
vet_minority                    12          4.897959
other_gender_minority            2          0.816327


In [39]:
#Adding a column for binary values if the row/respondent indicated at least one minority feature.
#Specifying which columns to include in the binary count -- female not included.
cols_to_include = ['race_minority', 'other_gender_minority', 'sex_minority', 'religion_minority', \
                  'vet_minority', 'dis_minority', 'age_minority']

#Adding the binary values in the new column
diversity_df['binary_diversity'] = diversity_df[cols_to_include].apply(lambda row: 1 if row.any() == 1 else 0, axis=1)

In [40]:
#Adding a column for binary values if the row/respondent indicated at least one minority feature.
#Specifying which columns to include in the binary count -- female is included.
cols_to_include = ['female', 'race_minority', 'other_gender_minority', 'sex_minority', 'religion_minority', \
                  'vet_minority', 'dis_minority', 'age_minority']

#Adding the binary values to the new column
diversity_df['binary_diversity_with_female'] = diversity_df[cols_to_include].apply \
(lambda row: 1 if row.any() == 1 else 0, axis=1)

In [41]:
#Adding a column for the sum of minority features reported
#Specifying which columns to include in the binary count -- female not included.
cols_to_include = ['race_minority', 'other_gender_minority', 'sex_minority', 'religion_minority', \
                  'vet_minority', 'dis_minority', 'age_minority']

#Adding the ordinal values to the new column
diversity_df['sum_diversity'] = diversity_df[cols_to_include].sum(axis=1)

In [42]:
#Adding a column for the sum of minority features reported.
#Specifying which columns to include in the binary count -- female not included.
cols_to_include = ['female', 'race_minority', 'other_gender_minority', 'sex_minority', 'religion_minority', \
                  'vet_minority', 'dis_minority', 'age_minority']

#Adding the ordinal values to the new column
diversity_df['sum_diversity_with_female'] = diversity_df[cols_to_include].sum(axis=1)

In [43]:
#Adding a new column with ordinal values of 0 = no minority features reported, 1 = one reported, and 2 = 2+ reported
diversity_df['three_diversity'] = diversity_df['sum_diversity'].apply(lambda x: 1 if x in [1] else 2 if x in \
                                                                      [2, 3, 4, 5] else 0)

#Adding an index column for future use
diversity_df['IDX'] = diversity_df.index

In [44]:
#Adding another column with ordinal values for 0, 1, or 2+ minority features reported -- including female
diversity_df['three_diversity_with_female'] = diversity_df['sum_diversity_with_female'].apply \
(lambda x: 1 if x in [1] else 2 if x in [2, 3, 4, 5] else 0)

diversity_df['IDX'] = diversity_df.index

In [45]:
# Function to calculate count and percentage of 1s for each column
#NOTE: can disregard the sum_diversity and three_diversity columns here (and see next cells below).
def binary_count_and_percentage(diviersity_df):
    count_of_ones = diversity_df.sum()  # Count of 1s
    percentage_of_ones = (count_of_ones / diversity_df.count()) * 100  # Percentage of 1s
    div_desc_df = pd.DataFrame({
        'Count of 1s': count_of_ones,
        'Percentage of 1s': percentage_of_ones
    })
    return div_desc_df.sort_values(by='Count of 1s', ascending=False)  # Sort by count in descending order

# Call the function
div_desc_df = binary_count_and_percentage(diversity_df)

# Display the result
print(div_desc_df)

                              Count of 1s  Percentage of 1s
IDX                                 29890      12200.000000
sum_diversity_with_female             556        226.938776
three_diversity_with_female           434        177.142857
sum_diversity                         348        142.040816
three_diversity                       306        124.897959
binary_diversity_with_female          240         97.959184
female                                208         84.897959
binary_diversity                      207         84.489796
religion_minority                     135         55.102041
age_minority                           77         31.428571
race_minority                          52         21.224490
sex_minority                           42         17.142857
dis_minority                           28         11.428571
vet_minority                           12          4.897959
other_gender_minority                   2          0.816327


In [46]:
#Code to return the count and percentage of the sum_diversity column
def get_count_percentage(diversity_df, sum_diversity):
    
# Get counts of each unique value
    counts = diversity_df[sum_diversity].value_counts()
    
# Calculate the percentage
    percentages = diversity_df[sum_diversity].value_counts(normalize=True) * 100
    
# Combine counts and percentages into a DataFrame
    sum_div_desc_df = pd.DataFrame({
        'Count': counts,
        'Percentage': percentages
    })
    
    return sum_div_desc_df

sum_div_desc_df = get_count_percentage(diversity_df, 'sum_diversity')

print(sum_div_desc_df)

   Count  Percentage
1    108   44.081633
2     62   25.306122
0     38   15.510204
3     33   13.469388
4      3    1.224490
5      1    0.408163


In [47]:
#Code to return the count and percentage of the sum_diversity_with_female column
def get_count_percentage(diversity_df, sum_diversity_with_female):
    
# Get counts of each unique value
    counts = diversity_df[sum_diversity_with_female].value_counts()
    
# Calculate the percentage
    percentages = diversity_df[sum_diversity_with_female].value_counts(normalize=True) * 100
    
# Combine counts and percentages into a DataFrame
    sum_div_female_desc_df = pd.DataFrame({
        'Count': counts,
        'Percentage': percentages
    })
    
    return sum_div_female_desc_df

sum_div_female_desc_df = get_count_percentage(diversity_df, 'sum_diversity_with_female')

print(sum_div_female_desc_df)

   Count  Percentage
2    104   42.448980
3     62   25.306122
1     46   18.775510
4     24    9.795918
0      5    2.040816
5      4    1.632653


In [48]:
#Code to return the count and percentage of the three_diversity column
def get_count_percentage(diversity_df, three_diversity):
    
# Get counts of each unique value
    counts = diversity_df[three_diversity].value_counts()
    
# Calculate the percentage
    percentages = diversity_df[three_diversity].value_counts(normalize=True) * 100
    
# Combine counts and percentages into a DataFrame
    three_div_desc_df = pd.DataFrame({
        'Count': counts,
        'Percentage': percentages
    })
    
    return three_div_desc_df

three_div_desc_df = get_count_percentage(diversity_df, 'three_diversity')

print(three_div_desc_df)

   Count  Percentage
1    108   44.081633
2     99   40.408163
0     38   15.510204


In [49]:
#Code to return the count and percentage of the three_diversity_with_female column
def get_count_percentage(diversity_df, three_diversity_with_female):
    
# Get counts of each unique value
    counts = diversity_df[three_diversity_with_female].value_counts()
    
# Calculate the percentage
    percentages = diversity_df[three_diversity_with_female].value_counts(normalize=True) * 100
    
# Combine counts and percentages into a DataFrame
    three_div_female_desc_df = pd.DataFrame({
        'Count': counts,
        'Percentage': percentages
    })
    
    return three_div_female_desc_df

three_div_female_desc_df = get_count_percentage(diversity_df, 'three_diversity_with_female')

print(three_div_female_desc_df)

   Count  Percentage
2    194   79.183673
1     46   18.775510
0      5    2.040816


In [50]:
#Merging multiple dataframes to have all possible IVs and the diversity dataframe in one dataframe
#Specifying the dataframes to include
dfs = [ceo_df, org_size_df, service_settings_df, geo_df, position_category_df, position_duration_df, \
       degree_ordinal_df, training_df, certifications_df, diversity_df]

# Code to merge 
merged_div_df= dfs[0] # Creating new object equivalent to first df - ceo_df
for df in dfs[1:]: # For every item in the df list, starting with org_size_df and iterate
    merged_div_df=pd.merge(merged_div_df, df, on='IDX', how='right') # New object is merged based on index colum, right join

merged_div_df

,ceo_behavior_analysis,ceo_business,ceo_education,ceo_finance,ceo_psychology,ceo_human_resources,ceo_organizational_behavior_management,ceo_other_allied,ceo_other_other,ceo_unknown,IDX,org_size,clinic_outpatient,community,home,hospital_inpatient,residential_group_home,school,vocational_program,other_setting,setting_unknown,midwest_us,south_us,west_us,northeast_us,other_geo,geo_unknown,board_member,case_manager,director,direct_service,executive_member,position_other,position_unknown,position_duration,degree_ordinal,ed_behavior_analysis,ed_business,ed_education,ed_finance,ed_hr,ed_obm,ed_psychology,ed_social_work,ed_domain_other,ed_domain_unknown,bcba,cpa,slp,lba,licensed_psychologist,licensed_sw,other_rbt_bcaba,other_cert_teaching,other_cert,no_cert,cert_unknown,female,race_minority,other_gender_minority,sex_minority,religion_minority,vet_minority,dis_minority,age_minority,binary_diversity,binary_diversity_with_female,sum_diversity,sum_diversity_with_female,three_diversity,three_diversity_with_female
0,0,0,0,0,0,0,0,0,1,0,0,4,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,2,3,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,1,3,4,2,2
1,1,1,1,0,1,0,0,0,0,0,1,2,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,3,3,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,2,2,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,2,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,3,4,2,2
3,1,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,2,3,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,2,1,2
4,0,1,0,0,0,0,0,0,0,0,4,4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,3,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,3,4,2,2
5,0,1,0,0,0,0,0,0,0,0,5,4,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,2,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,2,1,2
6,1,0,0,0,0,0,0,0,0,0,6,7,0,1,1,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,2,3,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,3,3,2,2
7,1,0,0,0,1,0,0,0,0,0,7,2,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,3,3,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,3,3,2,2
8,1,0,1,0,1,0,0,0,0,0,8,8,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,2,1,2
9,0,0,0,0,0,0,0,1,0,0,9,2,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,2,2,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,2,2,2,2


In [51]:
#DAVID TO LOOK AT THIS CELL !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#Calculating a critical correlation threshold rather than the .3 conservative estimate (website z-score)
#What are the benefits/limitations for using the exact vs. conservative estimate for the stat sig threshold? 

import numpy as np
import scipy.stats as stats

# Function to calculate the critical correlation threshold
def critical_correlation_threshold(sample_size, alpha=0.05):
    # Degrees of freedom
    df = sample_size - 2

    # Critical t-value for a two-tailed test
    t_critical = stats.t.ppf(1 - alpha / 2, df)

    # Convert t-value to correlation threshold
    r_critical = t_critical / np.sqrt(t_critical**2 + df)

    return r_critical

# Adding my values
sample_size = 245  
alpha = 0.05

# Compute critical correlation threshold
r_threshold = critical_correlation_threshold(sample_size, alpha)
print(f"Critical correlation threshold for sample size {sample_size} (alpha={alpha}): {r_threshold:.4f}")

Critical correlation threshold for sample size 245 (alpha=0.05): 0.1254


In [52]:
#DAVID TO LOOK AT THIS CELL TOO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Conduct correlation analyses with a critical threshold check
def correlation_analysis_with_threshold(merged_div_df, alpha=0.05):
    results = []
    columns = df.columns
    sample_size = len(merged_div_df)

    # Compute critical correlation threshold
    r_threshold = critical_correlation_threshold(sample_size, alpha)

    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col1, col2 = columns[i], columns[j]
            x, y = merged_div_df[col1], merged_div_df[col2]

            # Calculate Pearson correlation
            corr_coef, p_value = stats.pearsonr(x, y)

            # Append results
            results.append({
                "Column 1": col1,
                "Column 2": col2,
                "Correlation Coefficient": corr_coef,
                "P-value": p_value,
                "Significant by P-value": p_value < alpha,
                "Significant by Threshold": abs(corr_coef) >= r_threshold
            })

    return pd.DataFrame(results)

# Run the modified analysis
correlation_results_with_threshold = correlation_analysis_with_threshold(merged_div_df, alpha=.05)

correlation_results_with_threshold

,Column 1,Column 2,Correlation Coefficient,P-value,Significant by P-value,Significant by Threshold
0,female,race_minority,-0.059851,3.508896e-01,False,False
1,female,other_gender_minority,-0.215101,7.004355e-04,True,True
2,female,sex_minority,-0.050121,4.348033e-01,False,False
3,female,religion_minority,0.077639,2.259479e-01,False,False
4,female,vet_minority,-0.009916,8.772729e-01,False,False
5,female,dis_minority,-0.099295,1.211173e-01,False,False
6,female,age_minority,-0.131893,3.911999e-02,True,True
7,female,binary_diversity,-0.023263,7.171180e-01,False,False
8,female,binary_diversity_with_female,0.342224,3.882712e-08,True,True
9,female,sum_diversity,-0.122432,5.565040e-02,False,False


In [53]:
####I'm HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#Check my list of correlations (word document) against the list below (old, .3 cut-off)
#Do I need/want to make any changes to the modeling dataframes???
#What are the benefits/limitations for using the exact vs. conservative estimate for the stat sig threshold? 

# Creating a correlation matrix
correlation_matrix = merged_div_df.corr()

# Filter the correlation matrix to get values greater than or less than the threshold determined above
significant_corr = correlation_matrix[(correlation_matrix > 0.1254) | (correlation_matrix < -0.1254)]

# Remove self-correlations (correlation of a column with itself)
significant_corr = significant_corr.where(~pd.np.eye(significant_corr.shape[0], dtype=bool))

# Get the list of column pairs with significant correlations
significant_pairs = significant_corr.stack().reset_index()
significant_pairs.columns = ['Column 1', 'Column 2', 'Correlation']

# Sort by absolute correlation value (optional)
significant_pairs = significant_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

# Counting number of observations present across both columns being correlated
count_list = []
for row in range(len(significant_pairs)):
    Col1 = significant_pairs['Column 1'][row]
    Col2 = significant_pairs['Column 2'][row]
    temp_df = merged_div_df[[Col1, Col2]]
    temp_df['count'] = temp_df.sum(axis=1)
    temp_df = temp_df[temp_df['count'] == 2]
    count_list.append(len(temp_df))

significant_pairs['count'] = count_list

significant_pairs

#?????????????????????????????????????????? Should I keep this information (here or somewhere else???)
#10-20% would be 24 to 48+
#Paper -- we made that up :) the number that could impact modeling


#positive correlations:
#sum_diversity and three_diversity -- see modeling plan below 
#binary_diversity and three_diversity -- see modeling plan below
#sum_diversity_with_female and three_diversity_with_female -- see modeling plan below
#binary_diversity and sum_diversity -- see modeling plan
#sex_minority and sum_diversity -- see modeling plan
#binary_diversity_with_female and three_diversity_with_female -- see modeling plan
#sum_diversity_with_female and sex_minority -- see modeling plan
#religion_minority and #sum_diversity_with_female -- see modeling plan
#religion_minority and binary_diversity -- see modeling plan
#religion_minority and sum_diversity -- see modeling plan
#sex_minority and three_diversity -- see modeling plan
#ed_behavior_analysis and bcba -- omit ed_behavior_analysis from all modeling df
#dis_minority and sum_diversity -- see modeling plan
#sum_diversity and race_minority -- see modeling plan
#community and home -- omit community from all modeling df
#dis_minority and sum_diversity_with_female -- see modeling plan
#sum_diversity_with_female and race_minority -- see modeling plan
#race_minority and three_diversity -- see modeling plan
#age_minority and three_diversity -- see modeling plan
#three_diversity_with_female and female -- see modeling plan
#age_minority and sum_diversity -- see modeling plan
#binary_diversity_with_female and female -- see modeling plan
#community and school -- omit community from all modeling df 
#sum_diversity_with_female and binary_diversity_with_female -- see modeling plan
#bcba and lba -- omit lba from all modeling df
#dis_minority and three_diversity -- see modeling plan
#position_duration and age_minority - omit position_duration from all modeling df
#org_size and northeast_us -- omit org_size from all modeling df


#negative correlation:
#ceo_behavior_analys and org_size -- omit org_size for all modeling df
#degree_ordinal and other_rbt_bcaba -- omit other_rbt_bcaba for all modeling df 




#MY PLANS FOR MODELING FOR THE MULTICOLLINIARITY ABOVE: 
#One df with all minority columns (and none of the diversity aggregate columns)
#One df with no minority columns (except the female column) and binary_diversity column
#One df with no minority columns and binary_diversity_with_female column (and no female column)
#One df with no minority columns (except the female column) and sum_diversity column
#One df with no minority columns and sum_diversity_with_female column (and no female column)
#One df with no minority columns (except the femal column) and three_diversity column
#One df with no minority columns and three_diversity_with_female column (and no female column)


C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:13: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  significant_corr = significant_corr.where(~pd.np.eye(significant_corr.shape[0], dtype=bool))
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['count'] = temp_df.sum(axis=1)
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['count'] = temp_df.sum(axis=1)
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['count'] = temp_df.sum(axis=1)
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['count'] = temp_df.sum(axis=1)
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['count'] = temp_df.sum(axis=1)
C:\Users\krglodowski\AppData\Local\Temp\ipykernel_19924\2097055303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,Column 1,Column 2,Correlation,count
0,setting_unknown,geo_unknown,1.000000,1
1,geo_unknown,setting_unknown,1.000000,1
2,ed_finance,ed_domain_unknown,1.000000,6
3,ed_domain_unknown,ed_finance,1.000000,6
4,sum_diversity,sum_diversity_with_female,0.933948,13
5,sum_diversity_with_female,sum_diversity,0.933948,13
6,ceo_finance,ceo_unknown,0.926928,13
7,ceo_unknown,ceo_finance,0.926928,13
8,three_diversity,sum_diversity,0.912834,108
9,sum_diversity,three_diversity,0.912834,108


In [54]:
#Is there a difference in female between groups for: geo_region? position? degree? ceo background? org size? 
# All the same as above but for the various aggregate diversity measures? 

# merged_div_df -- the df with all possible IVs, minority columns, and diversity aggregate columns

#How do I describe this one?????????????????????? and is the above important to keep (here or somewhere else?)
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# Target column to compare against other categorical variables
target_col = 'female'

# List of other columns to compare
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")

##?????????????????????????????????????????????????? keep these (here or somewhere else??)  
#executive_member vs female - but adjusted p-value is .18
#ceo_other_allied vs female - but adjusted p-value is .09


Contingency Table for org_size vs female:
female     0   1
org_size        
1          5  22
2         13  67
3          6  47
4          4  32
5          1  18
6          5  13
7          3   4
8          0   5

Row-normalized proportions for org_size vs female:
female           0         1
org_size                    
1         0.185185  0.814815
2         0.162500  0.837500
3         0.113208  0.886792
4         0.111111  0.888889
5         0.052632  0.947368
6         0.277778  0.722222
7         0.428571  0.571429
8         0.000000  1.000000

Chi-Square Statistic for org_size vs female: 9.324109382313356
P-Value: 0.23021623854643836
Degrees of Freedom: 7

Contingency Table for degree_ordinal vs female:
female           0    1
degree_ordinal         
-1               1    8
 0               0    1
 1               2   12
 2              20  149
 3              14   38

Row-normalized proportions for degree_ordinal vs female:
female                 0         1
degree_ordinal      

In [55]:
# Target column to compare against other categorical variables with chi-squared analyses
target_col = 'three_diversity'

# List of other columns to compare 
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")
    

#ceo_business vs. three_diversity - but adjusted p-value is .86
#midwest_us vs. three_diversity - but adjusted p-value is .21
#west_us vs. three_diversity - bud adjusted p-value is .14



Contingency Table for org_size vs three_diversity:
three_diversity   0   1   2
org_size                   
1                 4  10  13
2                 6  37  37
3                15  23  15
4                 7  15  14
5                 1  12   6
6                 4   6   8
7                 1   2   4
8                 0   3   2

Row-normalized proportions for org_size vs three_diversity:
three_diversity         0         1         2
org_size                                     
1                0.148148  0.370370  0.481481
2                0.075000  0.462500  0.462500
3                0.283019  0.433962  0.283019
4                0.194444  0.416667  0.388889
5                0.052632  0.631579  0.315789
6                0.222222  0.333333  0.444444
7                0.142857  0.285714  0.571429
8                0.000000  0.600000  0.400000

Chi-Square Statistic for org_size vs three_diversity: 18.082717433932523
P-Value: 0.20303938975991098
Degrees of Freedom: 14

Contingency Table fo

In [56]:
# Target column to compare against other categorical variables
target_col = 'sum_diversity'

# List of other columns to compare
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")

###???????????????????????????????????????? keep (here or somewhere else????)
#ceo_business vs sum_diversity - but adjusted p-value is .67


Contingency Table for org_size vs sum_diversity:
sum_diversity   0   1   2   3  4  5
org_size                           
1               4  10   9   3  1  0
2               6  37  23  13  0  1
3              15  23  10   4  1  0
4               7  15   9   5  0  0
5               1  12   3   2  1  0
6               4   6   6   2  0  0
7               1   2   1   3  0  0
8               0   3   1   1  0  0

Row-normalized proportions for org_size vs sum_diversity:
sum_diversity         0         1         2         3         4       5
org_size                                                               
1              0.148148  0.370370  0.333333  0.111111  0.037037  0.0000
2              0.075000  0.462500  0.287500  0.162500  0.000000  0.0125
3              0.283019  0.433962  0.188679  0.075472  0.018868  0.0000
4              0.194444  0.416667  0.250000  0.138889  0.000000  0.0000
5              0.052632  0.631579  0.157895  0.105263  0.052632  0.0000
6              0.222222  0.

In [57]:
# Target column to compare against other categorical variables with chi-squared analyses
target_col = 'binary_diversity'

# List of other columns to compare
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")

##?????????????????????????????????????????????? keep (here or somewhere else??)
#ceo_business and binary_diversity - but adjusted p-value is .25
#midwest_us and binary_diversity - bud adjusted p-value is .07


Contingency Table for org_size vs binary_diversity:
binary_diversity   0   1
org_size                
1                  4  23
2                  6  74
3                 15  38
4                  7  29
5                  1  18
6                  4  14
7                  1   6
8                  0   5

Row-normalized proportions for org_size vs binary_diversity:
binary_diversity         0         1
org_size                            
1                 0.148148  0.851852
2                 0.075000  0.925000
3                 0.283019  0.716981
4                 0.194444  0.805556
5                 0.052632  0.947368
6                 0.222222  0.777778
7                 0.142857  0.857143
8                 0.000000  1.000000

Chi-Square Statistic for org_size vs binary_diversity: 13.167574317729896
P-Value: 0.06812921885538858
Degrees of Freedom: 7

Contingency Table for degree_ordinal vs binary_diversity:
binary_diversity   0    1
degree_ordinal           
-1                 1    8
 0

In [58]:
# Target column to compare against other categorical variables with chi-squared analyses
target_col = 'binary_diversity_with_female'

# List of other columns to compare
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")
    
####????????????????????????????????????????????? keep (here or somewhere else??)
#midwest_us vs binary_diversity_with_female - but adjusted p-value is .09


Contingency Table for org_size vs binary_diversity_with_female:
binary_diversity_with_female  0   1
org_size                           
1                             1  26
2                             1  79
3                             1  52
4                             0  36
5                             0  19
6                             2  16
7                             0   7
8                             0   5

Row-normalized proportions for org_size vs binary_diversity_with_female:
binary_diversity_with_female         0         1
org_size                                        
1                             0.037037  0.962963
2                             0.012500  0.987500
3                             0.018868  0.981132
4                             0.000000  1.000000
5                             0.000000  1.000000
6                             0.111111  0.888889
7                             0.000000  1.000000
8                             0.000000  1.000000

Chi-Square

In [59]:
# Target column to compare against other categorical variables with chi-squared analyses
target_col = 'sum_diversity_with_female'

# List of other columns to compare
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")

#####??????????????????????????????????????????????????????? keep (here or somewhere else???)
#direct_service vs sum_diversity_with_female - but adjusted p-value is .57
#midwest_us vs sum_diversity_with_female - but adjusted p-value is .297
#west_us vs sum_diversity_with_female - but adjusted p-value is .059


Contingency Table for org_size vs sum_diversity_with_female:
sum_diversity_with_female  0   1   2   3  4  5
org_size                                      
1                          1   6   7  10  2  1
2                          1   9  36  24  9  1
3                          1  15  25   8  3  1
4                          0   8  16   8  4  0
5                          0   1  12   4  1  1
6                          2   4   5   5  2  0
7                          0   3   0   2  2  0
8                          0   0   3   1  1  0

Row-normalized proportions for org_size vs sum_diversity_with_female:
sum_diversity_with_female         0         1         2         3         4  \
org_size                                                                      
1                          0.037037  0.222222  0.259259  0.370370  0.074074   
2                          0.012500  0.112500  0.450000  0.300000  0.112500   
3                          0.018868  0.283019  0.471698  0.150943  0.056604   
4 

In [60]:
# Target column to compare against other categorical variables with chi-squared analyses
target_col = 'three_diversity_with_female'

# List of other columns to compare 
columns_to_compare = ['org_size','degree_ordinal', 'case_manager', 'director', 'executive_member', \
                     'direct_service', 'ceo_behavior_analysis', 'ceo_business', 'ceo_psychology', \
                     'ceo_education', 'ceo_other_allied', 'midwest_us', 'south_us', 'west_us', 'northeast_us', \
                     'other_geo']

# Store p-values for Bonferroni correction later
p_values = []

# Loop through columns and perform chi-square tests
for col in columns_to_compare:
    # Create a contingency table
    contingency_table = pd.crosstab(merged_div_df[col], merged_div_df[target_col])
    
    # Normalize the proportions row-wise
    row_normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    
    print(f"\nContingency Table for {col} vs {target_col}:")
    print(contingency_table)
    
    print(f"\nRow-normalized proportions for {col} vs {target_col}:")
    print(row_normalized)

    # Run the Chi-Square test using statsmodels
    chi2_result = sm.stats.Table(contingency_table).test_nominal_association()

    # Output the test results
    print(f"\nChi-Square Statistic for {col} vs {target_col}: {chi2_result.statistic}")
    print(f"P-Value: {chi2_result.pvalue}")
    print(f"Degrees of Freedom: {chi2_result.df}")

    # Store p-values for Bonferroni correction
    p_values.append(chi2_result.pvalue)

# Apply Bonferroni correction
adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

# Output the adjusted p-values
for col, adj_p in zip(columns_to_compare, adjusted_p_values):
    print(f"\nAdjusted P-Value (Bonferroni) for {col} vs {target_col}: {adj_p}")

###########????????????????????????????????????????????? keep (here or somewhere else???)
#midwest_us vs three_diversity_with_female - the adjusted p-value is .042!!!!!!!!


Contingency Table for org_size vs three_diversity_with_female:
three_diversity_with_female  0   1   2
org_size                              
1                            1   6  20
2                            1   9  70
3                            1  15  37
4                            0   8  28
5                            0   1  18
6                            2   4  12
7                            0   3   4
8                            0   0   5

Row-normalized proportions for org_size vs three_diversity_with_female:
three_diversity_with_female         0         1         2
org_size                                                 
1                            0.037037  0.222222  0.740741
2                            0.012500  0.112500  0.875000
3                            0.018868  0.283019  0.698113
4                            0.000000  0.222222  0.777778
5                            0.000000  0.052632  0.947368
6                            0.111111  0.222222  0.666667
7        

In [61]:
# Creating a modeling df that has the possible IVs (minus the multicollinear ones) and individual minority columns
# Dropping the columns flagged for multicollinearity
initial_modeling_df = merged_div_df.drop(['ed_behavior_analysis', 'community', 'lba', 'position_duration', \
                                         'org_size', 'other_rbt_bcaba'], axis=1)

#Dropping the other diversity aggregate columns
modeling_df_minority = initial_modeling_df.drop(['binary_diversity', 'binary_diversity_with_female', \
                                                'sum_diversity', 'sum_diversity_with_female', 'three_diversity', \
                                                'three_diversity_with_female'], axis=1)

In [62]:
#creating a df below that has the initial multicollinearity columns dropped and also the minority columns dropped.
#This dataframe will be used to create separate modeling dataframes for each aggregate diversity column
aggregate_modeling_df = initial_modeling_df.drop(['race_minority', 'other_gender_minority', 'sex_minority', \
                                                  'religion_minority', 'vet_minority', 'dis_minority', \
                                                  'age_minority'], axis=1)

In [63]:
#Creating a modeling dataframe with all possible IVs and the binary diversity column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_binary = aggregate_modeling_df.drop(['binary_diversity_with_female', 'sum_diversity', \
                                                'sum_diversity_with_female', 'three_diversity', \
                                                'three_diversity_with_female'], axis=1)

In [64]:
#Creating a modeling dataframe with all possible IVs and the three_diversity column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_three = aggregate_modeling_df.drop(['binary_diversity', 'binary_diversity_with_female', 'sum_diversity', \
                                                'sum_diversity_with_female', 'three_diversity_with_female'], axis=1)

In [65]:
#Creating a modeling dataframe with all possible IVs and the sum_diversity column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_sum = aggregate_modeling_df.drop(['binary_diversity', 'binary_diversity_with_female', 'three_diversity', \
                                                'sum_diversity_with_female', 'three_diversity_with_female'], axis=1)

In [66]:
#Creating a modeling dataframe with all possible IVs and the binary_with_female column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_binary_with_female = aggregate_modeling_df.drop(['female', 'binary_diversity', 'sum_diversity', \
                                                             'three_diversity', 'sum_diversity_with_female', \
                                                             'three_diversity_with_female'], axis=1)

In [67]:
#Creating a modeling dataframe with all possible IVs and the three_with_female column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_three_with_female = aggregate_modeling_df.drop(['female', 'binary_diversity', 'sum_diversity', \
                                                             'three_diversity', 'sum_diversity_with_female', \
                                                             'binary_diversity_with_female'], axis=1)

In [68]:
#Creating a modeling dataframe with all possible IVs and the sum_with_female column 
#Dropping all other aggregate diversity columns in the aggregate modeling df
modeling_df_sum_with_female = aggregate_modeling_df.drop(['female', 'binary_diversity', 'sum_diversity', \
                                                             'three_diversity', 'three_diversity_with_female', \
                                                             'binary_diversity_with_female'], axis=1)

In [69]:
#Saving the modeling dataframes as csv files
#These include the decisions with the .3 estimate for the correlations (not the super precise ones)

modeling_df_sum_with_female.to_csv('modeling_df_sum_with_female.csv', index=False)

modeling_df_three_with_female.to_csv('modeling_df_three_with_female.csv', index=False)

modeling_df_binary_with_female.to_csv('modeling_df_binary_with_female.csv', index=False)

modeling_df_sum.to_csv('modeling_df_sum.csv', index=False)

modeling_df_three.to_csv('modeling_df_three.csv', index=False)

modeling_df_binary.to_csv('modeling_df_binary.csv', index=False)

modeling_df_minority.to_csv('modeling_df_minority.csv', index=False)
